# Working with 3rd party (detached) Prompts/Prompt Template Assets


This notebook should be run using with Runtime 22.2 & Python 3.10 or greater runtime environment, if you are viewing this in Watson Studio, and do not see Python 3.10.x in the upper right corner of your screen, please update the runtime now. 

The notebook will create a rag prompt template asset in a given project, configure OpenScale to monitor that PTA and evaluate generative quality metrics and model health metrics.

If users wish to execute this notebook for task types other than rag, please consult [this](https://github.com/IBM/watson-openscale-samples/blob/main/IBM%20Cloud/WML/notebooks/watsonx/README.md) document for guidance on evaluating prompt templates for the available task types.

Note : User can search for `EDIT THIS` and fill the inputs needed.

## Prerequisite

* It requires service credentials for IBM Watson OpenScale:
* Requires a CSV file containing the test data that needs to be evaluated
* Requires the ID of project in which you want to create the prompt template asset.

### Contents

- [Setup](#settingup)
- [Create Prompt template](#prompt)
- [Prompt Setup](#ptatsetup)
- [Risk evaluations for prompt template asset subscription](#evaluate)
- [Display the Model Risk metrics](#mrmmetric)
- [Display the Generative AI Quality metrics](#genaimetrics)
- [Plot rougel and rougelsum metrics against records](#plotproject)
- [See factsheets information](#factsheetsspace)

## Setup <a name="settingup"></a>

In [ ]:
!pip install --upgrade datasets==2.10.0 --no-cache | tail -n 1
!pip install --upgrade ibm-aigov-facts-client==1.0.111 --extra-index-url https://test.pypi.org/simple | tail -n 1
!pip install --upgrade ibm-watson-openscale | tail -n 1
!pip install --upgrade pydantic==1.10.11 --no-cache | tail -n 1

In [ ]:
!pip install chromadb==0.4.15 | tail -n 1
!pip install langchain==0.0.317 | tail -n 1
!pip install -U pypdf transformers InstructorEmbedding sentence_transformers==2.2.2 | tail -n 1
!pip install pysqlite3-binary dataclasses | tail -n 1
!pip install textstat | tail -n 1
!pip install --upgrade langchain-huggingface | tail -n 1
!pip install openai==0.28 | tail -n 1

In [1]:
import warnings
warnings.filterwarnings('ignore')

## Necessary LangChain Imports

In [4]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceInstructEmbeddings
from langchain.vectorstores import Chroma

In [3]:
chunk_size = 2000
chunk_overlap = 100

## Load the document for QnA

In [6]:
from ibm_watson_studio_lib import access_project_or_space
wslib = access_project_or_space()

In [7]:
wslib.download_file("BankFaq.pdf")

{'file_name': 'BankFaq.pdf', 'summary': ['loaded data', 'saved to file']}

In [8]:
loader = PyPDFLoader("BankFaq.pdf")
data = loader.load()

Ignoring wrong pointing object 6 0 (offset 0)
Ignoring wrong pointing object 8 0 (offset 0)
Ignoring wrong pointing object 10 0 (offset 0)
Ignoring wrong pointing object 12 0 (offset 0)
Ignoring wrong pointing object 14 0 (offset 0)
Ignoring wrong pointing object 16 0 (offset 0)
Ignoring wrong pointing object 32 0 (offset 0)


Ignoring wrong pointing object 12 0 (offset 0)


Ignoring wrong pointing object 14 0 (offset 0)


Ignoring wrong pointing object 16 0 (offset 0)


Ignoring wrong pointing object 32 0 (offset 0)


## Every page in pdf is counted as unique document

In [9]:
print (f'You have {len(data)} document(s) in your data')

You have 7 document(s) in your data


## Quick Peep into the document

In [10]:
print (f'There are {len(data[0].page_content)} characters in first page')
print(f"content of first page\n : {data[0].page_content}")

There are 2232 characters in first page
content of first page
 : Conditions of Failure: Digital Banking 1 My beneficiary's account is not yet credited with the amount I have transferred Please use 'raise a query' option available in support section in the app using your transaction reference number. You could also track your query from time to time.  You can now browse our 'Discover our features' videos available in support section of our app for a demo of various features available in our app.  2 From the list of beneficiaries, I am not able to see the name of beneficiary I have added earlier Please use 'raise a query' option available in support section in the app. You could also track your query from time to time. You can now browse our 'Discover our features' videos available in support section of our app for a demo of various features available in our app.  3 I have transferred an amount of $XXX to a beneficiary. But the transaction has been rejected and credited back to my accoun

## Split the document over chunk_size and chunk_overlap
More details about RecursiveCharacterTextSplitter here - https://python.langchain.com/docs/modules/data_connection/document_transformers/text_splitters/recursive_text_splitter

In [11]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size= chunk_size, chunk_overlap=chunk_overlap)
docs = text_splitter.split_documents(data)

In [12]:
print(f'We have total documents after split: {len(docs)}')

We have total documents after split: 9


## Load the model from HuggingFace for generating Embeddings

In [13]:
from langchain_huggingface import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

In [14]:
__import__('pysqlite3')
import sys
sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')

In [15]:
doc_db = Chroma.from_documents(docs, embedding=embeddings)

## Questions to ask to the model

In [17]:
query1 = 'From the list of beneficiaries, I am not able to see the name of beneficiary I have added earlier.'
query2 = 'If a transaction is not credited to the beneficiary account, does the remitter get back the money?'
query3 = 'What if bill amount is greater than autopay amount set?'
query4 = 'What should be done if customer does not find transaction older than 3 years?'
query5 = 'My Occupation is not in the given list. What should I do?'
query6 = 'The changes I have made in my profile are not being saved. What should I do?'
query7 = 'Merchant site is not responding.'
query8 = 'Is the Interest rate fixed or variable?'
query9 = 'Can I select my monthly payment amount?'
query10 = 'What do I need to enroll in Online Banking?'
query11 = 'How do I enroll in paperless statements in the mobile app?'
query12 = 'What are the risks in investing mutual funds?'

reference_answers = [
    'Please use \'raise a query\' option available in support section in the app. You could also track your query from time to time. You can now browse our \'Discover our features\' videos available in support section of our app.',
    'Yes. If the beneficiary\'s bank is unable to credit the beneficiary\'s account for any reason, the former will return the money to the remitting bank within 2 hours. Once the amount is received by the remitting bank, it is credited to the remitter\'s account by the branch concerned.',
    'Each bill for amount greater than the specified can only be paid against your specific authority.',
    'Please contact branch and put request for older transaction Or your request will be taken by us  and will be forwarded to our specialized team/Branch for resolution.',
    'Select Others and then write details',
    'User will be asked about the fields that have been modified and a query is raised to address this issue.',
    'Please contact the Merchant Customer Care for more details.',
    'Interest rate is fixed for the entire tenure of the Loan.',
    'You can select a combination of loan amount & tenure to suit your monthly payment.',
    'You will need your Social Security number or Tax ID, Email address, your account number OR ATM/debit card and PIN.',
    'Please go to your account settings, mobile and turn on paperless billing.',
    'Mutual funds are subject to market risk.'
]

question_texts = [query1, query2, query3, query4, query5, query6, query7, query8, query9, query10, query11, query12]

In [18]:
len(reference_answers)

12

## Get the relevant contexts

In [19]:
### Retrieve relevant context
# Fetch paragraphs similar to the question.
relevant_contexts = []

for question_text in question_texts:
    relevant_chunks = doc_db.similarity_search_with_score(query=question_text, k=1)
    relevant_contexts.append(relevant_chunks)

In [20]:
len(relevant_contexts)

12

In [21]:
class color:
   PURPLE = '\033[95m'
   CYAN = '\033[96m'
   DARKCYAN = '\033[36m'
   BLUE = '\033[94m'
   GREEN = '\033[92m'
   YELLOW = '\033[93m'
   RED = '\033[91m'
   BOLD = '\033[1m'
   UNDERLINE = '\033[4m'
   END = '\033[0m'

In [22]:
def get_str(entity, value):
    return (entity + ": " + str(value))

## Get the set of chunks for one of the questions.

In [23]:
sample_chunks = relevant_contexts[0]
for doc, score in sample_chunks:
    print("=========")
    #print(str(color.BOLD) + get_str("Paragraph index", doc.) + str(color.END))
    print(str(color.BOLD) + get_str("Distance", score) + str(color.END))
    print("\t"+ str(color.BOLD) + "Paragraph: " + str(color.END), doc.page_content)  

Distance: 1.0109853744506836
	Paragraph:  Conditions of Failure: Digital Banking 1 My beneficiary's account is not yet credited with the amount I have transferred Please use 'raise a query' option available in support section in the app using your transaction reference number. You could also track your query from time to time.  You can now browse our 'Discover our features' videos available in support section of our app for a demo of various features available in our app.  2 From the list of beneficiaries, I am not able to see the name of beneficiary I have added earlier Please use 'raise a query' option available in support section in the app. You could also track your query from time to time. You can now browse our 'Discover our features' videos available in support section of our app for a demo of various features available in our app.  3 I have transferred an amount of $XXX to a beneficiary. But the transaction has been rejected and credited back to my account. What could be the re

In [24]:
question_texts

['From the list of beneficiaries, I am not able to see the name of beneficiary I have added earlier.',
 'If a transaction is not credited to the beneficiary account, does the remitter get back the money?',
 'What if bill amount is greater than autopay amount set?',
 'What should be done if customer does not find transaction older than 3 years?',
 'My Occupation is not in the given list. What should I do?',
 'The changes I have made in my profile are not being saved. What should I do?',
 'Merchant site is not responding.',
 'Is the Interest rate fixed or variable?',
 'Can I select my monthly payment amount?',
 'What do I need to enroll in Online Banking?',
 'How do I enroll in paperless statements in the mobile app?',
 'What are the risks in investing mutual funds?']

## Make the prompt comprising of Prompt Instruction, Context and the Question

In [25]:
def make_prompt(context, question_text):
    prompt_text = '''
Context: {}
Answer the following question using only information from the above context. 
Answer in a complete sentence. If there is no good answer in the context, say "I don't know".
Question: {}
Answer:
'''
    formatted_prompt = prompt_text.format(context, question_text)
    return formatted_prompt
    #return f"Please answer the following question using the c.\n\n" + f"{context}\n" + f"{question_text}"

In [26]:
prompt_texts = []

In [27]:
def construct_context(relevant_context):
    context = ''
    for doc, score in relevant_context:
        context = context + doc.page_content + '\n\n'
    return context    

In [28]:
for relevant_context, question_text in zip(relevant_contexts, question_texts):
    print('query: '+question_text)
    context = construct_context(relevant_context)
    prompt_text = make_prompt(context, question_text)
    prompt_texts.append(prompt_text)

query: From the list of beneficiaries, I am not able to see the name of beneficiary I have added earlier.
query: If a transaction is not credited to the beneficiary account, does the remitter get back the money?
query: What if bill amount is greater than autopay amount set?
query: What should be done if customer does not find transaction older than 3 years?
query: My Occupation is not in the given list. What should I do?
query: The changes I have made in my profile are not being saved. What should I do?
query: Merchant site is not responding.
query: Is the Interest rate fixed or variable?
query: Can I select my monthly payment amount?
query: What do I need to enroll in Online Banking?
query: How do I enroll in paperless statements in the mobile app?
query: What are the risks in investing mutual funds?


In [29]:
#sample prompt text
print(prompt_texts[0])


Context: Conditions of Failure: Digital Banking 1 My beneficiary's account is not yet credited with the amount I have transferred Please use 'raise a query' option available in support section in the app using your transaction reference number. You could also track your query from time to time.  You can now browse our 'Discover our features' videos available in support section of our app for a demo of various features available in our app.  2 From the list of beneficiaries, I am not able to see the name of beneficiary I have added earlier Please use 'raise a query' option available in support section in the app. You could also track your query from time to time. You can now browse our 'Discover our features' videos available in support section of our app for a demo of various features available in our app.  3 I have transferred an amount of $XXX to a beneficiary. But the transaction has been rejected and credited back to my account. What could be the reason? The reason for reversal of

In [30]:
len(prompt_texts)

12

## Run the prompt against Azure OpenAI GPT35 Turbo model

In [31]:
import os
import openai

In [32]:
openai.api_type = "azure"
openai.api_base = "https://azure-openai-xxxx-xxxxx.openai.azure.com/"
openai.api_version = "2023-09-15-preview"
openai.api_key = 'xxxxxx'

In [33]:
def get_completion(prompt_text):
    response = openai.Completion.create(
        engine = "azure-openai-deployment-001",
        prompt=prompt_text,
        temperature=0.1,
        max_tokens=100,
        top_p=0.5,
        frequency_penalty=0,
        presence_penalty=0,
        stop='\n'
    )
    return response.choices[0].text
    #return response

In [34]:
results = []

for prompt_text in prompt_texts:
    model_response = get_completion(prompt_text=prompt_text)
    results.append(model_response)

In [35]:
results

["Please use 'raise a query' option available in support section in the app. You could also track your query from time to time. You can now browse our 'Discover our features' videos available in support section of our app for a demo of various features available in our app. ",
 "Yes. If the beneficiary's bank is unable to credit the beneficiary's account for any reason, the former will return the money to the remitting bank within 2 hours. Once the amount is received by the remitting bank, it is credited to the remitter's account by the branch concerned.",
 'Each bill for amount greater than the specified can only be paid against your specific authority. ',
 'Please contact branch and put request for older transaction Or your request will be taken by us  and will be forwarded to our specialized team/Branch for resolution.',
 'Select Others and then write details',
 'User will be asked about the fields that have been modified and a query is raised to address this issue.',
 'Please conta

In [36]:
final_payload = []
for question, relevant_context, answer, reference_answer in zip(question_texts, relevant_contexts, results, reference_answers):
    record_payload = []

    context = construct_context(relevant_context)
    record_payload.append(context)
    record_payload.append(question)
    record_payload.append(answer)
    record_payload.append(reference_answer)
    final_payload.append(record_payload)

In [37]:
import pandas as pd
llm_data = pd.DataFrame(final_payload, columns=['context', 'question', 'generated_text', 'reference_answer'])
llm_data.head()

,context,question,generated_text,reference_answer
0,Conditions of Failure: Digital Banking 1 My be...,"From the list of beneficiaries, I am not able ...",Please use 'raise a query' option available in...,Please use 'raise a query' option available in...
1,Conditions of Failure: Digital Banking 1 My be...,If a transaction is not credited to the benefi...,Yes. If the beneficiary's bank is unable to cr...,Yes. If the beneficiary's bank is unable to cr...
2,45 I have not received any letter asking my co...,What if bill amount is greater than autopay am...,Each bill for amount greater than the specifie...,Each bill for amount greater than the specifie...
3,Please use smart search option for old transac...,What should be done if customer does not find ...,Please contact branch and put request for olde...,Please contact branch and put request for olde...
4,7 What is the next step after raising the quer...,My Occupation is not in the given list. What s...,Select Others and then write details,Select Others and then write details


In [38]:
len(llm_data)

12

In [39]:
test_data_path = "banking_faq_data.csv"
llm_data.to_csv(test_data_path, index=False)

Note: you may need to restart the kernel to use updated packages.

### Provision services and configure credentials

If you have not already, provision an instance of IBM Watson OpenScale using the [OpenScale link in the Cloud catalog](https://cloud.ibm.com/catalog/services/watson-openscale).

Your Cloud API key can be generated by going to the [**Users** section of the Cloud console](https://cloud.ibm.com/iam#/users). From that page, click your name, scroll down to the **API Keys** section, and click **Create an IBM Cloud API key**. Give your key a name and click **Create**, then copy the created key and paste it below.

**NOTE:** You can also get OpenScale `API_KEY` using IBM CLOUD CLI.

How to install IBM Cloud (bluemix) console: [instruction](https://console.bluemix.net/docs/cli/reference/ibmcloud/download_cli.html#install_use)

How to get api key using console:
```
bx login --sso
bx iam api-key-create 'my_key'
```

In [40]:
CPD_URL = "https://cpd-cpd-instance.apps.xxxxx.cp.fyre.ibm.com"
CPD_USERNAME = "cpadmin"
CPD_PASSWORD = "xxxx"
WOS_SERVICE_INSTANCE_ID = "00000000-0000-0000-0000-000000000000" # If None, default instance would be used

## Set the project ID

In order to set up a development type subscription, the PTA must be within the project. Please supply the project ID where the PTA needs to be created.

In [41]:
PROJECT_ID = "cb9ebb21-177f-41cd-a5bf-1b950fe5e95e" # YOUR_PROJECT_ID

## Function to create the access token

This function generates an IAM access token using the provided credentials. The API calls for creating and scoring prompt template assets utilize the token generated by this function.

In [42]:
import requests, json
def generate_access_token():
    headers={}
    headers["Content-Type"] = "application/json"
    headers["Accept"] = "application/json"
    data = {
        "username": CPD_USERNAME,
        "password": CPD_PASSWORD
    }
    data = json.dumps(data).encode("utf-8")
    url = CPD_URL + "/icp4d-api/v1/authorize"
    response = requests.post(url=url, data=data, headers=headers, verify=False)
    json_data = response.json()
    iam_access_token = json_data["token"]
        
    return iam_access_token

iam_access_token = generate_access_token()

# Create Prompt template <a name="prompt"></a>

Create a prompt template for a rag task

In [43]:
from ibm_aigov_facts_client import AIGovFactsClient

from ibm_aigov_facts_client import CloudPakforDataConfig
creds=CloudPakforDataConfig(
    service_url=CPD_URL,
    username=CPD_USERNAME,
    password=CPD_PASSWORD
)
facts_client = AIGovFactsClient(
    cloud_pak_for_data_configs=creds,
    container_id=PROJECT_ID,
    container_type="project",
    disable_tracing=True
)

In [44]:
# facts_client.assets.delete_prompt_asset(asset_id='ccadaf85-83dc-449d-990d-7b3136e8397f')

In [45]:
from ibm_aigov_facts_client import DetachedPromptTemplate, PromptTemplate
detached_information = DetachedPromptTemplate(
    prompt_id="banking_faq_prompt",
    model_id="gpt-35-turbo",
    model_provider="Azure OpenAI GPT35",
    model_name="GPT-3.5 Turbo",
    model_url="https://learn.microsoft.com/en-us/azure/ai-services/openai/concepts/models#gpt-35",
    prompt_url="prompt_url",
    prompt_additional_info={"Region": "eastus"}
)

In [46]:
task_id = "retrieval_augmented_generation"
name = "Banking FAQ External prompt (Azure OpenAI GPT35)"
description = "Banking FAQ detached prompt"
model_id = "gpt-35-turbo"

# define parameters for PromptTemplate
prompt_variables = {"context": "", "question": ""}
input = """Context: {context}
        Answer the following question using only information from the above context. 
        Answer in a complete sentence. If there is no good answer in the context, say "I don't know".
        Question: {question}
        Answer:"""
input_prefix= ""
output_prefix= ""

In [47]:
prompt_template = PromptTemplate(
    input=input,
    prompt_variables=prompt_variables,
    input_prefix=input_prefix,
    output_prefix=output_prefix,
    
    #model parameters
    decoding_method="greedy",
    repetition_penalty=1,
    max_new_tokens= 30,
    stop_sequences=['\n'],
    temperature=0.1,
    top_p=0.5
)

In [48]:
pta_details = facts_client.assets.create_detached_prompt(
    model_id=model_id,
    task_id=task_id,
    name=name,
    description=description,
    prompt_details=prompt_template,
    detached_information=detached_information
)
project_pta_id = pta_details.to_dict()["asset_id"]

2024/06/20 01:11:37 INFO : The prompt with ID 9aef2762-4289-4dcf-8580-d1d20bc60cdf was created successfully in container_id cb9ebb21-177f-41cd-a5bf-1b950fe5e95e.


# Prompt setup <a name="ptatsetup"></a>

### Configure OpenScale

In [49]:
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator, CloudPakForDataAuthenticator

from ibm_watson_openscale import *
from ibm_watson_openscale.supporting_classes.enums import *
from ibm_watson_openscale.supporting_classes import *

authenticator = CloudPakForDataAuthenticator(
    url=CPD_URL,
    username=CPD_USERNAME,
    password=CPD_PASSWORD,
    disable_ssl_verification=True
)
wos_client = APIClient(
    service_url=CPD_URL,
    authenticator=authenticator,
    service_instance_id=WOS_SERVICE_INSTANCE_ID
)
data_mart_id = wos_client.service_instance_id
print(wos_client.version)

3.0.38


### Openscale instance mapping with the project

When the authentication is on CPD then we need to add additional step of mapping the project_id/space_id to an OpenScale instance.

In [50]:
from ibm_watson_openscale.base_classes import ApiRequestFailure

try:
  wos_client.wos.add_instance_mapping(                
    service_instance_id=data_mart_id,
    project_id=PROJECT_ID
  )
except ApiRequestFailure as arf:
   if arf.response.status_code == 409:
      # Instance mapping already exists
      pass
   else:
      raise arf

### Setup the prompt template asset in project for evaluation with supported monitor dimensions

The prompt template assets from project is only supported with `development` operational space ID. Running the below cell will create a development type subscription from the prompt template asset created within the project.

The available parameters that can be passed for `execute_prompt_setup` function are:

 * `prompt_template_asset_id` : Id of prompt template asset for which subscription needs to be created.
 * `label_column` :  The name of the column containing the ground truth or actual labels.
 * `project_id` : The GUID of the project.
 * `space_id` : The GUID of the space.
 * `deployment_id` : (optional) The GUID of the deployment.
 * `operational_space_id` : The rank of the environment in which the monitoring is happening. Accepted values are `development`, `pre_production`, `production`.
 * `problem_type` : (optional) The task type to monitor for the given prompt template asset.
 * `classification_type` : The classification type `binary`/`multiclass` applicable only for `classification` problem (task) type.
 * `input_data_type` : The input data type.
 * `supporting_monitors` : Monitor configuration for the subscription to be created.
 * `background_mode` : When `True`, the promt setup operation will be executed in the background

In [51]:
label_column = "reference_answer"
operational_space_id = "development"
problem_type = "retrieval_augmented_generation"
input_data_type = "unstructured_text"
monitors = {
    "generative_ai_quality": {
        "parameters": {
            "min_sample_size": 10,
            "metrics_configuration": {
                "faithfulness": {
                    #"attributions_count": 3,
                    #"ngrams": 2,
                },
                "answer_relevance": {},
                "rouge_score": {},
                "exact_match": {},
                "bleu": {},
                "unsuccessful_requests": {
                    #"unsuccessful_phrases": []
                }
            }
        }
    }
}

In [52]:
response = wos_client.monitor_instances.mrm.execute_prompt_setup(
    prompt_template_asset_id=project_pta_id, 
    project_id=PROJECT_ID,
    label_column=label_column,
    context_fields = ["context"],
    question_field = "question",
    operational_space_id=operational_space_id, 
    problem_type=problem_type,
    input_data_type=input_data_type, 
    supporting_monitors=monitors, 
    background_mode=False
)
result = response.result
result.to_dict()

This method will be deprecated in the next release and be replaced by wos_client.wos.execute_prompt_setup() method



 Waiting for end of adding prompt setup 9aef2762-4289-4dcf-8580-d1d20bc60cdf 




running.
finished

---------------------------------------------------------------
 Successfully finished setting up prompt template subscription 
---------------------------------------------------------------




{'prompt_template_asset_id': '9aef2762-4289-4dcf-8580-d1d20bc60cdf',
 'project_id': 'cb9ebb21-177f-41cd-a5bf-1b950fe5e95e',
 'deployment_id': 'cb831032-a2a2-4235-af2d-082b7e42d2f9',
 'service_provider_id': 'f5f2edab-0d12-49ba-820d-d25bdccab14a',
 'subscription_id': 'd1c633da-5970-4635-8672-e8ec379a197c',
 'mrm_monitor_instance_id': 'e41e7cd9-1f43-4df7-a42e-b10810f9ce12',
 'start_time': '2024-06-20T01:11:40.032398Z',
 'end_time': '2024-06-20T01:11:52.161759Z',
 'status': {'state': 'FINISHED'}}

With the below cell, users can  read the  prompt setup task and check its status

In [53]:
response = wos_client.monitor_instances.mrm.get_prompt_setup(
    prompt_template_asset_id=project_pta_id,
    project_id=PROJECT_ID
)
result = response.result
result_json = result.to_dict()

if result_json["status"]["state"] == "FINISHED":
    print("Finished prompt setup. The response is {}".format(result_json))
else:
    print("Prompt setup failed. The response is {}".format(result_json))

This method will be deprecated in the next release and be replaced by wos_client.wos.get_prompt_setup() method
Finished prompt setup. The response is {'prompt_template_asset_id': '9aef2762-4289-4dcf-8580-d1d20bc60cdf', 'project_id': 'cb9ebb21-177f-41cd-a5bf-1b950fe5e95e', 'deployment_id': 'cb831032-a2a2-4235-af2d-082b7e42d2f9', 'service_provider_id': 'f5f2edab-0d12-49ba-820d-d25bdccab14a', 'subscription_id': 'd1c633da-5970-4635-8672-e8ec379a197c', 'mrm_monitor_instance_id': 'e41e7cd9-1f43-4df7-a42e-b10810f9ce12', 'start_time': '2024-06-20T01:11:40.032398Z', 'end_time': '2024-06-20T01:11:52.161759Z', 'status': {'state': 'FINISHED'}}


### Read required IDs from prompt setup response

In [54]:
subscription_id = result_json["subscription_id"]
mrm_monitor_instance_id = result_json["mrm_monitor_instance_id"]

## Show all the monitor instances of the production subscription
The following cell lists the monitors present in the development subscription along with their respective statuses and other details. Please wait for all the monitors to be in active state before proceeding further.

In [55]:
wos_client.monitor_instances.show(target_target_id=subscription_id)

00000000-0000-0000-0000-000000000000,active,d1c633da-5970-4635-8672-e8ec379a197c,subscription,generative_ai_quality,2024-06-20 01:11:44.149000+00:00,2023fca5-64a6-4abd-b8fa-0b74eeecacf2
00000000-0000-0000-0000-000000000000,active,d1c633da-5970-4635-8672-e8ec379a197c,subscription,model_health,2024-06-20 01:11:44.682000+00:00,92efc0cb-f363-42e3-bf28-d949c592c53b
00000000-0000-0000-0000-000000000000,active,d1c633da-5970-4635-8672-e8ec379a197c,subscription,mrm,2024-06-20 01:11:45.282000+00:00,e41e7cd9-1f43-4df7-a42e-b10810f9ce12


# Risk evaluations for PTA subscription <a name="evaluate"></a>

### Evaluate the prompt template subscription

For the risk assessment of a development type subscription the user needs to have an evaluation dataset. The risk evaluation function takes the evaluation dataset path as a parameter for evaluation of the configured metric dimensions. If there is a discrepancy between the feature columns in the subscription and the column names in the uploading CSV, users has the option to supply a mapping JSON file to associate the CSV column names with the feature column names in the subscription.


**Note:* If you are running this notebook from Watson studio, you may first need to upload your test data to studio and run code snippet to download feedback data file from project to local directory

The following cell will assess the test data with the subscription of the prompt template asset and produce relevant measurements for the configured monitor.

In [56]:
test_data_set_name = "data"
content_type = "multipart/form-data"
body = {}

In [57]:
# # Preparing the test data, removing extra columns
# cols_to_remove = ["uid", "doc", "title", "id"]
# llm_data=llm_data.rename(columns={"answer": "generated_text"})
# for col in cols_to_remove:
#     if col in llm_data:
#         del llm_data[col]
# llm_data.to_csv(test_data_path, index=False)

In [58]:
llm_data.columns

Index(['context', 'question', 'generated_text', 'reference_answer'], dtype='object')

In [59]:
response  = wos_client.monitor_instances.mrm.evaluate_risk(
    monitor_instance_id=mrm_monitor_instance_id,
    test_data_set_name=test_data_set_name, 
    test_data_path=test_data_path,
    content_type=content_type,
    body=body,
    project_id=PROJECT_ID,
    includes_model_output=True,
    background_mode=False
)




 Waiting for risk evaluation of MRM monitor e41e7cd9-1f43-4df7-a42e-b10810f9ce12 




upload_in_progress
running..........
finished

---------------------------------------
 Successfully finished evaluating risk 
---------------------------------------




### Read the risk evaluation response

After initiating the risk evaluation, the evaluation results are now available for review

In [60]:
response  = wos_client.monitor_instances.mrm.get_risk_evaluation(mrm_monitor_instance_id, project_id=PROJECT_ID)
response.result.to_dict()

{'metadata': {'id': '12f2d625-25ca-42c4-85d9-303902b88393',
  'created_at': '2024-06-20T01:12:05.601Z',
  'created_by': 'internal-service'},
 'entity': {'triggered_by': 'user',
  'parameters': {'evaluation_start_time': '2024-06-20T01:11:58.686235Z',
   'evaluator_user_key': '0119daae-ff3a-430d-9fbe-0e9ac263a963',
   'facts': {'state': 'finished'},
   'is_auto_evaluated': False,
   'measurement_id': 'd1e6be48-7b2b-4191-af9f-870e54179d41',
   'project_id': 'cb9ebb21-177f-41cd-a5bf-1b950fe5e95e',
   'prompt_template_asset_id': '9aef2762-4289-4dcf-8580-d1d20bc60cdf',
   'user_iam_id': '1000331001',
   'wos_created_deployment_id': 'cb831032-a2a2-4235-af2d-082b7e42d2f9',
   'publish_metrics': 'false',
   'evaluation_tests': ['drift_v2',
    'generative_ai_quality',
    'model_health',
    'quality']},
  'status': {'state': 'finished',
   'queued_at': '2024-06-20T01:12:05.595000Z',
   'started_at': '2024-06-20T01:12:06.106000Z',
   'updated_at': '2024-06-20T01:13:11.901000Z',
   'completed_at

# Display the Model Risk metrics <a name="mrmmetric"></a>

Having calculated the measurements for the Foundation Model subscription, the MRM metrics generated for this subscription are now available for your review.

In [61]:
wos_client.monitor_instances.show_metrics(monitor_instance_id=mrm_monitor_instance_id, project_id=PROJECT_ID)

2024-06-20 01:12:05.625000+00:00,tests_passed,d1e6be48-7b2b-4191-af9f-870e54179d41,1.0,None,None,['test_data_set_name:data'],mrm,e41e7cd9-1f43-4df7-a42e-b10810f9ce12,12f2d625-25ca-42c4-85d9-303902b88393,subscription,d1c633da-5970-4635-8672-e8ec379a197c
2024-06-20 01:12:05.625000+00:00,tests_run,d1e6be48-7b2b-4191-af9f-870e54179d41,1.0,None,None,['test_data_set_name:data'],mrm,e41e7cd9-1f43-4df7-a42e-b10810f9ce12,12f2d625-25ca-42c4-85d9-303902b88393,subscription,d1c633da-5970-4635-8672-e8ec379a197c
2024-06-20 01:12:05.625000+00:00,tests_skipped,d1e6be48-7b2b-4191-af9f-870e54179d41,2.0,None,None,['test_data_set_name:data'],mrm,e41e7cd9-1f43-4df7-a42e-b10810f9ce12,12f2d625-25ca-42c4-85d9-303902b88393,subscription,d1c633da-5970-4635-8672-e8ec379a197c
2024-06-20 01:12:05.625000+00:00,tests_failed,d1e6be48-7b2b-4191-af9f-870e54179d41,0.0,None,None,['test_data_set_name:data'],mrm,e41e7cd9-1f43-4df7-a42e-b10810f9ce12,12f2d625-25ca-42c4-85d9-303902b88393,subscription,d1c633da-5970-4635-8672-e8ec379a197c


# Display the Generative AI Quality metrics <a name="genaimetrics"></a>



Monitor instance ID of Generative ai quality metrics is required for reading its metrics.

In [62]:
monitor_definition_id = "generative_ai_quality"
result = wos_client.monitor_instances.list(
    data_mart_id=data_mart_id,
    monitor_definition_id=monitor_definition_id,
    target_target_id=subscription_id,
    project_id=PROJECT_ID
).result
result_json = result._to_dict()
genaiquality_monitor_id = result_json["monitor_instances"][0]["metadata"]["id"]
genaiquality_monitor_id

'2023fca5-64a6-4abd-b8fa-0b74eeecacf2'

Displaying the GenAIQ monitor metrics generated through the risk evaluation.

In [63]:
wos_client.monitor_instances.show_metrics(monitor_instance_id=genaiquality_monitor_id, project_id=PROJECT_ID)

2024-06-20 01:12:51.137561+00:00,rouge2,5782886e-6768-4689-b7b9-3731adf6549a,0.8977,None,None,"['computed_on:feedback', 'field_type:subscription', 'aggregation_type:mean']",generative_ai_quality,2023fca5-64a6-4abd-b8fa-0b74eeecacf2,ca30eafb-d923-4eb9-b3f7-3dc71ca2c25a,subscription,d1c633da-5970-4635-8672-e8ec379a197c
2024-06-20 01:12:51.137561+00:00,faithfulness,5782886e-6768-4689-b7b9-3731adf6549a,0.7353,None,None,"['computed_on:feedback', 'field_type:subscription', 'aggregation_type:mean']",generative_ai_quality,2023fca5-64a6-4abd-b8fa-0b74eeecacf2,ca30eafb-d923-4eb9-b3f7-3dc71ca2c25a,subscription,d1c633da-5970-4635-8672-e8ec379a197c
2024-06-20 01:12:51.137561+00:00,records_processed,5782886e-6768-4689-b7b9-3731adf6549a,12.0,None,None,"['computed_on:feedback', 'field_type:subscription', 'aggregation_type:mean']",generative_ai_quality,2023fca5-64a6-4abd-b8fa-0b74eeecacf2,ca30eafb-d923-4eb9-b3f7-3dc71ca2c25a,subscription,d1c633da-5970-4635-8672-e8ec379a197c
2024-06-20 01:12:51.137561+00:00,rougelsum,5782886e-6768-4689-b7b9-3731adf6549a,0.8989,None,None,"['computed_on:feedback', 'field_type:subscription', 'aggregation_type:mean']",generative_ai_quality,2023fca5-64a6-4abd-b8fa-0b74eeecacf2,ca30eafb-d923-4eb9-b3f7-3dc71ca2c25a,subscription,d1c633da-5970-4635-8672-e8ec379a197c
2024-06-20 01:12:51.137561+00:00,answer_relevance,5782886e-6768-4689-b7b9-3731adf6549a,0.3731,None,None,"['computed_on:feedback', 'field_type:subscription', 'aggregation_type:mean']",generative_ai_quality,2023fca5-64a6-4abd-b8fa-0b74eeecacf2,ca30eafb-d923-4eb9-b3f7-3dc71ca2c25a,subscription,d1c633da-5970-4635-8672-e8ec379a197c
2024-06-20 01:12:51.137561+00:00,bleu,5782886e-6768-4689-b7b9-3731adf6549a,0.9008,None,None,"['computed_on:feedback', 'field_type:subscription', 'aggregation_type:mean']",generative_ai_quality,2023fca5-64a6-4abd-b8fa-0b74eeecacf2,ca30eafb-d923-4eb9-b3f7-3dc71ca2c25a,subscription,d1c633da-5970-4635-8672-e8ec379a197c
2024-06-20 01:12:51.137561+00:00,rougel,5782886e-6768-4689-b7b9-3731adf6549a,0.8989,None,None,"['computed_on:feedback', 'field_type:subscription', 'aggregation_type:mean']",generative_ai_quality,2023fca5-64a6-4abd-b8fa-0b74eeecacf2,ca30eafb-d923-4eb9-b3f7-3dc71ca2c25a,subscription,d1c633da-5970-4635-8672-e8ec379a197c
2024-06-20 01:12:51.137561+00:00,exact_match,5782886e-6768-4689-b7b9-3731adf6549a,0.5,None,None,"['computed_on:feedback', 'field_type:subscription', 'aggregation_type:mean']",generative_ai_quality,2023fca5-64a6-4abd-b8fa-0b74eeecacf2,ca30eafb-d923-4eb9-b3f7-3dc71ca2c25a,subscription,d1c633da-5970-4635-8672-e8ec379a197c
2024-06-20 01:12:51.137561+00:00,unsuccessful_requests,5782886e-6768-4689-b7b9-3731adf6549a,0.0833,None,None,"['computed_on:feedback', 'field_type:subscription', 'aggregation_type:mean']",generative_ai_quality,2023fca5-64a6-4abd-b8fa-0b74eeecacf2,ca30eafb-d923-4eb9-b3f7-3dc71ca2c25a,subscription,d1c633da-5970-4635-8672-e8ec379a197c
2024-06-20 01:12:51.137561+00:00,rouge1,5782886e-6768-4689-b7b9-3731adf6549a,0.8989,None,None,"['computed_on:feedback', 'field_type:subscription', 'aggregation_type:mean']",generative_ai_quality,2023fca5-64a6-4abd-b8fa-0b74eeecacf2,ca30eafb-d923-4eb9-b3f7-3dc71ca2c25a,subscription,d1c633da-5970-4635-8672-e8ec379a197c


Note: First 10 records were displayed.


## Display record level metrics for Generative AI Quality 

Read the dataset id for generative ai quality dataset

In [64]:
result = wos_client.data_sets.list(
    target_target_id=subscription_id,
    target_target_type="subscription",
    type="gen_ai_quality_metrics"
).result

genaiq_dataset_id = result.data_sets[0].metadata.id
genaiq_dataset_id

'7ac1e27a-fc14-4b83-9ad1-3ca2dfa808ea'

Displaying record level metrics for generative ai quality

In [65]:
wos_client.data_sets.show_records(data_set_id=genaiq_dataset_id)

0.8837,0.9464,MRM_bd3fcbb9-c056-4ea8-8904-b520b115735c-0,feedback,2024-06-20T01:11:59.964Z,0.8864,0.1074,"{'faithfulness': 0.9464, 'faithfulness_attributions': [{'attributions': [{'faithfulness_scores': [0.8989, 0.8878, 0.8214], 'feature_name': 'context', 'feature_values': [""Conditions of Failure: Digital Banking 1 My beneficiary's account is not yet credited with the amount I have transferred Please use 'raise a query' option available in support section in the app using your transaction reference number. You could also track your query from time to time."", ""2 From the list of beneficiaries, I am not able to see the name of beneficiary I have added earlier Please use 'raise a query' option available in support section in the app. You could also track your query from time to time."", ""You can now browse our 'Discover our features' videos available in support section of our app for a demo of various features available in our app. 2 From the list of beneficiaries, I am not able to see the name of beneficiary I have added earlier Please use 'raise a query' option available in support section in the app.""]}], 'faithfulness_score': 0.8989, 'output_text': ""Please use 'raise a query' option available in support section in the app.""}, {'attributions': [{'faithfulness_scores': [0.9835, 0.9819, 0.9699], 'feature_name': 'context', 'feature_values': [""2 From the list of beneficiaries, I am not able to see the name of beneficiary I have added earlier Please use 'raise a query' option available in support section in the app. You could also track your query from time to time."", ""Conditions of Failure: Digital Banking 1 My beneficiary's account is not yet credited with the amount I have transferred Please use 'raise a query' option available in support section in the app using your transaction reference number. You could also track your query from time to time."", ""You could also track your query from time to time. You can now browse our 'Discover our features' videos available in support section of our app for a demo of various features available in our app.""]}], 'faithfulness_score': 0.9835, 'output_text': 'You could also track your query from time to time.'}, {'attributions': [{'faithfulness_scores': [0.9567, 0.9567, 0.9194], 'feature_name': 'context', 'feature_values': [""You could also track your query from time to time. You can now browse our 'Discover our features' videos available in support section of our app for a demo of various features available in our app."", ""You could also track your query from time to time. You can now browse our 'Discover our features' videos available in support section of our app for a demo of various features available in our app."", ""You can now browse our 'Discover our features' videos available in support section of our app for a demo of various features available in our app. You can now browse our 'Discover our features' videos available in support section of our app for a demo of various features available in our app.""]}], 'faithfulness_score': 0.9567, 'output_text': ""You can now browse our 'Discover our features' videos available in support section of our app for a demo of various features available in our app.""}]}",0.7966806751538719,ca30eafb-d923-4eb9-b3f7-3dc71ca2c25a,0.8864,0.0,0.0,0.8864
1.0,0.9044,MRM_bd3fcbb9-c056-4ea8-8904-b520b115735c-1,feedback,2024-06-20T01:11:59.964Z,1.0,0.88,"{'faithfulness': 0.9044, 'faithfulness_attributions': [{'attributions': [{'faithfulness_scores': [0.7508, 0.6957, 0.4594], 'feature_name': 'context', 'feature_values': ['4 If a transaction is not credited to the beneficiary account, does the remitter get back the money? Yes.', ""Yes. If the beneficiary's bank is unable to credit the beneficiary's account for any reason, the former will return the money to the remitting bank within 2 hours."", ""If the beneficiary's bank is unable to credit the beneficiary's account for any reason, the former will return the money to the remitting bank within 2 hour

# See factsheets information <a name="factsheetsspace"></a>

In [66]:
factsheets_url = f"{CPD_URL}/wx/prompt-details/{project_pta_id}/factsheet?context=wx&project_id={PROJECT_ID}"
print(f"User can navigate to the published facts in project {factsheets_url}")

User can navigate to the published facts in project https://cpd-cpd-instance.apps.wos414nfs2678.cp.fyre.ibm.com/wx/prompt-details/9aef2762-4289-4dcf-8580-d1d20bc60cdf/factsheet?context=wx&project_id=cb9ebb21-177f-41cd-a5bf-1b950fe5e95e


## Track the prompt in a usecase

In [67]:
ai_usecase_id = "96cb0c5c-7595-430f-bf04-1087e8178a2c"

In [68]:
ai_usecase = facts_client.assets.get_ai_usecase(
    ai_usecase_id=ai_usecase_id, 
    catalog_id="5c9d1701-e5d2-49d6-950f-f49f59bddbd0")

2024/06/20 01:15:55 INFO : Current AI usecase information: {'model_usecase_id': '96cb0c5c-7595-430f-bf04-1087e8178a2c', 'container_type': 'catalog', 'catalog_id': '5c9d1701-e5d2-49d6-950f-f49f59bddbd0', 'facts_type': 'model_entry_user'}


In [69]:
pta_obj = facts_client.assets.get_prompt(
    asset_id=project_pta_id, 
    container_type="project", 
    container_id=PROJECT_ID)

2024/06/20 01:15:57 INFO : Current prompt information: {'asset_id': '9aef2762-4289-4dcf-8580-d1d20bc60cdf', 'container_type': 'project', 'container_id': 'cb9ebb21-177f-41cd-a5bf-1b950fe5e95e', 'facts_type': 'modelfacts_user'}


In [70]:
approach_name = "Baking FAQ Application Monitoring"

## Remove the existing approach

In [71]:
approaches = ai_usecase.get_approaches()
for approach in approaches:
    _approach_name = approach.get_name()
    print(_approach_name)
    if _approach_name == approach_name:
        approach_id = approach.get_id()
        approach = ai_usecase.get_approach(approach_id)
        ai_usecase.remove_approach(approach)

2024/06/20 01:16:02 INFO : Approaches retrieved successfully
Default approach


In [72]:
next_steps_gen_approach = ai_usecase.create_approach(
    name=approach_name, 
    description="approach for next steps task")

2024/06/20 01:16:05 INFO : Approach created successfully


In [74]:
pta_obj.track(
    usecase=ai_usecase, 
    approach=next_steps_gen_approach, 
    version_number="minor", 
    version_comment="prompt approach for next steps generation task")

## Congratulations!

You have finished the hands-on lab for IBM Watson OpenScale. You can now navigate to the prompt template asset in your project and click on the Evaluate tab to visualize the results on the UI.